In [12]:
from tensorflow import keras

In [2]:
import keras
print(keras.__version__)


2.13.1


In [11]:
import tensorflow as tf
from tensorflow import keras
import os

# Define file paths
weights_dir = r"C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\MobNetV3Small_Kaggle\weights"
output_dir = r"C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_MLiSards"

# Classification weights
clf_weights_path = os.path.join(weights_dir, "fintuning_training_mobnetv3small_classification_weights.weights.keras")
# Alternative classification weights
clf_weights_alt_path = os.path.join(weights_dir, "50epochs_frozentraining_classification_mobnetv3small.weights.keras")

# Regression weights
reg_weights_path = os.path.join(weights_dir, "regressionfintuning_50epochs_mobnetv3small_weights.weights.keras")
# Alternative regression weights
reg_weights_alt_path = os.path.join(weights_dir, "regression50epochs_frozentraining_mobnetv3small.weights.keras")

# Output file paths
clf_h5_path = os.path.join(output_dir, "H5finetuned_50epochs_mobnetv3small_classification_model.h5")
reg_h5_path = os.path.join(output_dir, "H5regression_finetuned_50epochs_mobnetv3small_model.h5")

def create_classification_model(input_shape=(224, 224, 3), dropoutrate=0.2, num_classes=1):
    """Recreate the exact classification model architecture"""
    mobnetv3small = tf.keras.applications.MobileNetV3Small(
        weights='imagenet',  # This is important to match the original weights
        include_top=False,
        input_shape=input_shape
    )
    
    model = tf.keras.Sequential([
        mobnetv3small,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='sigmoid')
    ])
    
    # Freeze the base model layers
    mobnetv3small.trainable = False
    
    return model

def create_regression_model(input_shape=(224, 224, 3), dropoutrate=0.2, num_outputs=1):
    """Recreate the exact regression model architecture"""
    mobnetv3small = tf.keras.applications.MobileNetV3Small(
        weights='imagenet',
        include_top=False,
        input_shape=input_shape
    )
    
    model = tf.keras.Sequential([
        mobnetv3small,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dropout(dropoutrate),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(num_outputs, activation='linear')  # Linear activation for regression
    ])
    
    # Freeze the base model layers
    mobnetv3small.trainable = False
    
    return model

def process_model(create_fn, weights_path, output_path, model_type):
    """Create model, load weights, and save in H5 format"""
    try:
        print(f"Creating {model_type} model...")
        model = create_fn()
        
        print(f"Loading weights from {weights_path}...")
        try:
            # Try to load weights
            model.load_weights(weights_path)
            print("Weights loaded successfully!")
        except Exception as e:
            print(f"Error loading weights: {e}")
            return False
        
        print(f"Saving {model_type} model to {output_path}...")
        model.save(output_path, save_format="h5")
        print(f"{model_type} model saved successfully in H5 format!")
        return True
    except Exception as e:
        print(f"Error processing {model_type} model: {e}")
        return False

# Process classification model
print("=" * 50)
print("PROCESSING CLASSIFICATION MODEL")
print("=" * 50)
clf_success = process_model(
    create_classification_model, 
    clf_weights_path, 
    clf_h5_path, 
    "classification"
)

if not clf_success:
    print("Trying alternative classification weights...")
    clf_success = process_model(
        create_classification_model, 
        clf_weights_alt_path, 
        clf_h5_path, 
        "classification"
    )

# Process regression model
print("\n" + "=" * 50)
print("PROCESSING REGRESSION MODEL")
print("=" * 50)
reg_success = process_model(
    create_regression_model, 
    reg_weights_path, 
    reg_h5_path, 
    "regression"
)

if not reg_success:
    print("Trying alternative regression weights...")
    reg_success = process_model(
        create_regression_model, 
        reg_weights_alt_path, 
        reg_h5_path, 
        "regression"
    )

print("\n" + "=" * 50)
print("RESULTS SUMMARY")
print("=" * 50)
print(f"Classification model: {'SUCCESS' if clf_success else 'FAILED'}")
print(f"Regression model: {'SUCCESS' if reg_success else 'FAILED'}")

PROCESSING CLASSIFICATION MODEL
Creating classification model...
4334752/4334752 [==============================] - 1s 0us/step
Loading weights from C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\MobNetV3Small_Kaggle\weights\fintuning_training_mobnetv3small_classification_weights.weights.keras...
Error loading weights: axes don't match array
Trying alternative classification weights...
Creating classification model...
Loading weights from C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\MobNetV3Small_Kaggle\weights\50epochs_frozentraining_classification_mobnetv3small.weights.keras...
Weights loaded successfully!
Saving classification model to C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_MLiSards\H5finetuned_50epochs_mobnetv3small_classification_model.h5...


c:\Users\benat\miniconda3\envs\ncar_env1\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


classification model saved successfully in H5 format!

PROCESSING REGRESSION MODEL
Creating regression model...
Loading weights from C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\MobNetV3Small_Kaggle\weights\regressionfintuning_50epochs_mobnetv3small_weights.weights.keras...
Error loading weights: axes don't match array
Trying alternative regression weights...
Creating regression model...
Loading weights from C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\MobNetV3Small_Kaggle\weights\regression50epochs_frozentraining_mobnetv3small.weights.keras...
Weights loaded successfully!
Saving regression model to C:\Users\benat\OneDrive\0. MSc MLiS\0. GitHub Repositories\PICAR-autopilot\autopilot\models\BenTyler_MLiSards\H5regression_finetuned_50epochs_mobnetv3small_model.h5...
regression model saved successfully in H5 format!

RESULTS SUMMARY
Classification model: SUCCESS
Regression model: SUCCESS
